In [ ]:
######## THIS BOOK IS TO CREATE THE YEARSLY STATS USING DATA FROM THE NEW EXTRACTION TECHNIQUE ########
########## HOPEFULLY WILL BE ABLE TO COME UP WITH A FUNCTION THAT WORKS FOR MOST DIFFERENT SPORTS TYPES ##########

# WANT FINAL DATA TO BE VERY SIMALR TO THE DATA TABLES CREATED BY THE OLD FUNCTION THAT 
### USED THE RECORD BY RECORD PARSING METHOD

### THIS WILL USE THE DATA THAT IS BROUGH DOWN FOR AN ENTIRE SEASON WITH A SINGLE ROW FOR EVERY GAME
### WILL NEED TO FIND A STRATEGY TO REATE AGG STATS FOR HOME AND AWAY RESULTS AND STATS
## OLD METHOD USED DATA WHERE EACH GAME HAD 2 ROWS, ONE FROM THE HOME TEAM PERSPECTIVE AND ONE FROM THE AWAY TEAM PERSPECTIVE



In [ ]:
## Dependencies and Setup
## Dependencies and Setup

import pandas as pd
import numpy as np
import requests
import time
import json
import datetime
import os
import re
import tqdm as tqdm

In [ ]:
### URL BLOCK ###

# # example url for softball

# eaxmple_url = 'https://my.mhsaa.com/DesktopModules/MHSAA-Endpoint/handlers/ContestWithParticipantDepotSearch.ashx?Method=ScoreCenterSearch&StartDate=2023-01-26&EndDate=2023-06-26&Sport=GSB&Level=V'


## static peices of the url

url_a = 'https://my.mhsaa.com/DesktopModules/MHSAA-Endpoint/handlers/ContestWithParticipantDepotSearch.ashx?Method=ScoreCenterSearch&StartDate='

# length of url_a
print(len(url_a))

url_b = '&EndDate='
url_c = '&Sport='
url_d = '&Level=V'


### Date format ###
# YYYY-MM-DD

## academic year start and end dates
# Start August 1st
# End July 31st

# example date
# 2023-01-26


### Sport codes ###

# Ice Hockey (Boys) = BIH
# Football = BFB
# Baseball = BBA
# Softball = GSB

# Boys Basket Ball = BBB
# Girls Basket Ball = GBB

# Lacrosse (Boys) = BBL
# Lacrosse (Girls) = GGL
# Soccer (Boys) = BSO
# Soccer (Girls) = GSO

In [ ]:

# df = df_all

#################### BRING ALL THE CODE FROM EXTRACT DATA SPORT WORKBOOK INTO SINGLE FUNCTION ####################

## Define close and blowout games
def get_game_situation(row):
    sport = row['SportCode']  # Assuming the sport name is in the format 'BSO - Baseball'
    score_diff = abs(row['HostScore'] - row['AwayScore'])

    # Check if either score is None or NaN. If so, return 'Unknown'
    if pd.isnull(row['HostScore']) or pd.isnull(row['AwayScore']):
        return 'Unknown'
    
    if score_diff <= score_threasolds[sport]['close']:
        return 'Close'
    elif score_diff >= score_threasolds[sport]['blowout']:
        return 'Blowout'
    else:
        return 'Normal'

############ GAME LEVEL PREPROSSING ############

###### SIMPLE FUNCTIONS FROM THE PREVIOUS NOTEBOOKS ######

# construct the url for a given year
def url_construct(year):
    # Create start and end dates in teh YYYY-MM-DD format
    # Request one academic year of data at a time August 1st to July 31st
    start_date = str(year) + '-08-01'
    end_date = str(year + 1) + '-07-31'
    url = url_a + start_date + url_b + end_date + url_c + sport_code + url_d
    return url

# extrtact academic year from date
def get_year(date):
    return date[:4]


# If tournament name contains Regional, then regional; if quarterfinal, then quarterfinal, ect
def get_postseason_level(tournament_name):
    if tournament_name is not None:
        if "District" in tournament_name:
            return 'District'
        elif 'Regional' in tournament_name:
            return 'Regional'
        elif 'Quarter Final' in tournament_name:
            return 'Quarterfinal'
        elif 'Semifinal' in tournament_name:
            return 'Semifinal'
        elif 'Final' in tournament_name:
            return 'Final'
    # if not postseason, return none
    return None

############ START TO PROCESS THE DATA ############
######## SHAPES DF BY DROPPING AND RENAMING COLUMNS ########
#######  ADDS COULMN FOR SPORT NAME, YEAR, OT INFO AND FORIET INFO ########
def preprocess(df):

    # Creating ScoreNotes column
    df['ScoreNotes'] = None
    df.loc[df['Score'].str.contains('Forfeit'), 'ScoreNotes'] = 'Forfeit'
    df.loc[df['Score'].str.contains('Forfeit'), ['HostScore', 'AwayScore']] = 0

    # Creating and cleaning up OT column
    df['OT'] = df['Score'].str.extract(r'(\(.*\))') # extract anything surrounded by parentheses
    df['OT'] = df['OT'].str.replace('(', '')
    df['OT'] = df['OT'].str.replace(')', '')

    # Dropping the Score column
    df = df.drop(columns=['Score'])

    # Assuming that the rest of the columns are already correctly named, if not, you can rename them as follows
    df.rename(columns={
        'Season_Year': 'SeasonYear',
        'SportGenderDesc': 'SportName',
        'StartDate': 'StartDate',
        'IsPostSeason': 'IsPostSeason',
        'Notes': 'Notes',
        'HostSchoolId': 'HostSchoolId',
        'HostScore': 'HostScore',
        'HostTeamName': 'HostTeamName',
        'AwaySchoolId': 'AwaySchoolId',
        'AwayScore': 'AwayScore',
        'AwayTeamName': 'AwayTeamName',
        'ContestName': 'ContestNotes',
        'TournamentName': 'TournamentName',
        'TournamentNumber': 'TournamentNumber',
    }, inplace=True)

    df['PostSeasonLevel'] = df['TournamentName'].apply(get_postseason_level)
            # add sport code column
    

    df = df[['SeasonYear', 'SportName', 'StartDate', 'IsPostSeason', 'Notes', 
            'OT', 'ScoreNotes',
            'HostSchoolId', 'HostScore', 'HostTeamName', 
            'AwaySchoolId', 'AwayScore', 'AwayTeamName', 
            'ContestNotes',  
            'PostSeasonLevel','TournamentNumber']]
    

    return df


## Send the data to the preprocess function
# df = preprocess(df)


############ Add further processing to the data ############

def yearly_stats(df):
    # drop any forfieted games (score = 0)
    df = df.loc[(df['HostScore'] != 0) & (df['AwayScore'] != 0)]

    #### Drop null scores
    df = df.dropna(subset=['HostScore', 'AwayScore'])
    ## scores to int
    df['HostScore'] = df['HostScore'].astype(int)
    df['AwayScore'] = df['AwayScore'].astype(int)


    # Create new columns holding the result for the home and away teams
    df['HostResult'] = np.where(df['HostScore'] > df['AwayScore'], 'W', 
                                np.where(df['HostScore'] < df['AwayScore'], 'L', 
                                        np.where(df['HostScore'] == df['AwayScore'], 'T', None)))

    df['AwayResult'] = np.where(df['HostScore'] > df['AwayScore'], 'L', 
                                np.where(df['HostScore'] < df['AwayScore'], 'W', 
                                        np.where(df['HostScore'] == df['AwayScore'], 'T', None)))
    # Return the dataframe
    return df




# ### Append all the dataframes together and save to a new csv
# df_all = pd.DataFrame()
# for df, name in dataframes:
#     df_all = df_all.append(df)

In [ ]:
## Create a loop that goes through each sport and each year and creates a url to pull data from
# there will be empty responses for some of the years and sports so will need to create a way to deal with that

# create an empty list to store the urls
url_list = []


## create a list of the sports to be used in the url
sport_list = ['BBA','BIH', 'BFB', 'BSB', 'GSB', 'BBB', 'GBB', 'BBL', 'GGL', 'BSO', 'GSO']

# define the years to parse
year_list = list(range(2000, 2023))

# create a loop that goes through each sport and each academic year and creates a url to pull data from
# academic year is defined as starting on August 1st and ending on July 31st
for sport in tqdm.tqdm(sport_list):
    for year in year_list:
        # create the start and end dates for the url
        start_date = str(year) + '-08-01'
        end_date = str(year + 1) + '-07-31'
        # create the url
        url = url_a + start_date + url_b + end_date + url_c + sport + url_d
        
        url_list.append(url)

# check the length of the url list
len(url_list)


########### TEST BLOCK ############
## reduce url list to 10 urls to test the code
# url_list = url_list[0:30]

In [ ]:
############## BLOCK TO RUN SCRAPE PARSE ##################


# Create empty dataframe to store results
master_df = pd.DataFrame()

response_list = []
# Create a list to store the dataframes
df_list = []

for url in tqdm.tqdm(url_list):
    try:
        response = requests.get(url).json()
        response_list.append(response)

        df = pd.DataFrame(response)

        # Find the start of the 'ScoreCenterSearch&StartDate=' string in the url
        start_index = url.find('ScoreCenterSearch&StartDate=') + len('ScoreCenterSearch&StartDate=')

        # The next four characters are the year
        year = url[start_index:start_index+4]

        # The sport code is the value after '&Sport=' in the url
        sport_start_index = url.find('&Sport=') + len('&Sport=')
        sport_end_index = url.find('&', sport_start_index)
        sport = url[sport_start_index:sport_end_index]

        df['SportCode'] = sport
        df['SeasonYear'] = year

        df_pre = preprocess(df)
        df = yearly_stats(df_pre)

        # Append the dataframe to the list
        df_list.append(df)

    except Exception as e:
        print(f"Request to {url} failed with error {e}")
        response_list.append('no response')

# Concatenate all dataframes in the list
master_df = pd.concat(df_list, ignore_index=True)


In [ ]:
master_df.info()

## ## value counts for each sport and year
master_df.groupby(['SeasonYear']).size()



In [ ]:
### Save master_df for review

master_df.to_csv(f'../data/game_level/full_parse_all_sport_6_28_v3', index=False)

In [1]:
import pandas as pd

### Load data so I can skip the parse step
master_df = pd.read_csv(f'../data/game_level/full_parse_all_sport_6_28_v2')
## SportName value
# master_df['SportName'].value_counts()
# master_df.sample(10)

In [2]:
## Create a column for ScoreCode based on the SportName

# create a dictionary to map the sport name to the score code
sport_dict = {'Baseball': 'BBA', 
            'Basketball (Boys)': 'BBB', 
            'Basketball (Girls)': 'GBB',
            'Soccer (Boys)': 'BSO',
            'Soccer (Girls)': 'GSO',
            'Football': 'BFB',
            'Softball': 'GSB',
            'Lacrosse (Boys)': 'BBL',
            'Lacrosse (Girls)': 'GGL',          
            'Ice Hockey': 'BIH'
            }

# map the sport name to the sport code and add to the dataframe
master_df['SportCode'] = master_df['SportName'].map(sport_dict)

In [3]:
### rENAME MASTER_DF TO DF
df = master_df

df.columns

## sport code value counts
df['SportCode'].value_counts()

## Create a new column for subcategory of post season 
# Districts, Regional, State
def assign_postseason_sublevel(row):
    if row == 'District':
        return 'District'
    elif row == 'Regional':
        return 'Regional'
    elif row in ['Final', 'Semifinal', 'Quarterfinal']:
        return 'State'
    else:
        return None

df['PostSeasonSubLevel'] = df['PostSeasonLevel'].apply(assign_postseason_sublevel)


In [4]:
# df.sample(20)

In [5]:
####################### NEW CODE BLOCK ############################
########### THIS BLOCK DEFINES FUNCTIONS USED TO TURN GAME LEVEL DATA INTO SUMMARRY STATS #######################

### 
# Create a new column for SeasonType
df['SeasonType'] = df['IsPostSeason'].apply(lambda x: 'Postseason' if x else 'Regular Season')

## Dictionary to hold the score thresholds for each sport
score_thresholds = {
    'BIH': {"close": 1, "blowout": 5},
    'BFB': {"close": 7, "blowout": 21},
    'BBA': {"close": 3, "blowout": 8},
    'GSB': {"close": 3, "blowout": 8},
    'BBB': {"close": 3, "blowout": 15},
    'GBB': {"close": 3, "blowout": 15},
    'BBL': {"close": 2, "blowout": 8},
    'GGL': {"close": 2, "blowout": 8},
    'BSO': {"close": 1, "blowout": 5},
    'GSO': {"close": 1, "blowout": 5},
}

## Determin game type (Close vs Blowout)
def get_game_type(row):
    threshold = score_thresholds[row['SportCode']]
    diff = abs(row['HostScore'] - row['AwayScore'])
    if diff <= threshold['close']:
        return 'Close'
    elif diff >= threshold['blowout']:
        return 'Blowout'
    else:
        return 'Normal'
## Apply the function to the dataframe
df['GameType'] = df.apply(get_game_type, axis=1)


In [6]:
df.columns

Index(['SeasonYear', 'SportName', 'StartDate', 'IsPostSeason', 'Notes', 'OT',
       'ScoreNotes', 'HostSchoolId', 'HostScore', 'HostTeamName',
       'AwaySchoolId', 'AwayScore', 'AwayTeamName', 'ContestNotes',
       'PostSeasonLevel', 'TournamentNumber', 'HostResult', 'AwayResult',
       'SportCode', 'PostSeasonSubLevel', 'SeasonType', 'GameType'],
      dtype='object')

In [7]:
### COPY, flip and amend the DF so I can group by team name in the next block of code

# Duplicate the dataframe, swapping host and away team names and scores
df_flipped = df.copy()
df_flipped['TeamName'] = df['AwayTeamName']
df_flipped['OpponentName'] = df['HostTeamName']
df_flipped['TeamScore'] = df['AwayScore']
df_flipped['OpponentScore'] = df['HostScore']
df_flipped['Result'] = df['HostResult'].map({'W': 'L', 'L': 'W', 'T': 'T'})

# Append the flipped dataframe to the original dataframe
df['TeamName'] = df['HostTeamName']
df['OpponentName'] = df['AwayTeamName']
df['TeamScore'] = df['HostScore']
df['OpponentScore'] = df['AwayScore']
df['Result'] = df['HostResult']
df = pd.concat([df, df_flipped])

# Now you can group by 'TeamName' and calculate the aggregates


In [8]:
df.sample(10)

df.columns

Index(['SeasonYear', 'SportName', 'StartDate', 'IsPostSeason', 'Notes', 'OT',
       'ScoreNotes', 'HostSchoolId', 'HostScore', 'HostTeamName',
       'AwaySchoolId', 'AwayScore', 'AwayTeamName', 'ContestNotes',
       'PostSeasonLevel', 'TournamentNumber', 'HostResult', 'AwayResult',
       'SportCode', 'PostSeasonSubLevel', 'SeasonType', 'GameType', 'TeamName',
       'OpponentName', 'TeamScore', 'OpponentScore', 'Result'],
      dtype='object')

In [9]:
## Define the slices of the df that will be used to calculate the aggregatesq

# All games for overall stats
all_games = df

## Regular and Post Season
df_postseason = df[df['SeasonType'] == 'Postseason']
df_regular_season = df[df['SeasonType'] == 'Regular Season']

## Games that went to overtime ['OT'] contains 'OT'
## Games that went to overtime ['OT'] contains 'OT'
df_overtime = df[df['OT'].str.contains('OT', na=False)]


# Close games and blowout games
df_close_games = df[df['GameType'] == 'Close']
df_blowout_games = df[df['GameType'] == 'Blowout']

# District, Regional and state games
df_district_games = df[df['PostSeasonSubLevel'] == 'District']
df_regional_games = df[df['PostSeasonSubLevel'] == 'Regional']
df_state_games = df[df['PostSeasonSubLevel'] == 'State']


## Home / Away games (only regular season)
df_regular_season['IsHomeGame'] = df_regular_season['HostTeamName'] == df_regular_season['TeamName']
df_home_games = df_regular_season[df_regular_season['IsHomeGame'] == True]
df_away_games = df_regular_season[df_regular_season['IsHomeGame'] == False]



C:\Users\Justin\AppData\Local\Temp\ipykernel_14644\3803746622.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_regular_season['IsHomeGame'] = df_regular_season['HostTeamName'] == df_regular_season['TeamName']


In [10]:
## cREATE a list of the dataframes that make up the slices of data and print a report about each one

df_list = [all_games, df_postseason, df_regular_season, df_overtime, df_close_games, df_blowout_games, df_district_games, df_regional_games, df_state_games, df_home_games, df_away_games]
df_names = ['AllGames', 'Postseason', 'RegularSeason', 'Overtime', 'CloseGames', 'BlowoutGames', 'DistrictGames', 'RegionalGames', 'StateGames', 'HomeGames', 'AwayGames']


for df in df_list:
    print('******************************************')
    # print(f'Dataframe: {df_list_names[df_list.index(df)]}')
    # print(f'Dataframe Title: {df.title}')
    print(f'Number of rows: {len(df)}')
    # print(f'Number of columns: {len(df.columns)}')
    print(f'Number of unique teams: {len(df.TeamName.unique())}')

    ## 

******************************************
Number of rows: 870394
Number of unique teams: 2795
******************************************
Number of rows: 94902
Number of unique teams: 873
******************************************
Number of rows: 775492
Number of unique teams: 2794
******************************************
Number of rows: 16364
Number of unique teams: 1014
******************************************
Number of rows: 182732
Number of unique teams: 1784
******************************************
Number of rows: 387684
Number of unique teams: 2322
******************************************
Number of rows: 70918
Number of unique teams: 872
******************************************
Number of rows: 16536
Number of unique teams: 735
******************************************
Number of rows: 3330
Number of unique teams: 467
******************************************
Number of rows: 387745
Number of unique teams: 2241
******************************************
Number of rows: 3

In [11]:
########## NEW BLOCK TO DO THE SUMMARY STATS ######################

# calcs each df in the list and adds it to a new list
# concatonates the list of dfs into a new df

summary_dfs = []  # List to store the summary DataFrames

for df, df_name in zip(df_list, df_names):
    grouped = df.groupby(['SeasonYear', 'SportCode', 'TeamName'])
    summary_df = grouped.agg({
        'HostSchoolId': 'count',  # Total games played
        'Result': ['count', lambda x: (x == 'W').sum(), lambda x: (x == 'L').sum(), lambda x: (x == 'T').sum()],  # Total wins, losses, and ties
        'TeamScore': ['mean', 'sum'],  # Average and total points for
        'OpponentScore': ['mean', 'sum'],  # Average and total points against
    })

    # Rename the columns
    summary_df.columns = ['TotalGames', 'ResultTotal', 'Wins', 'Losses', 'Ties', 'AvgPointsFor', 'TotalPointsFor', 'AvgPointsAgainst', 'TotalPointsAgainst']

    # Add other statistics
    summary_df['WinPct'] = summary_df['Wins'] / summary_df['TotalGames']
    summary_df['AvgMargin'] = summary_df['AvgPointsFor'] - summary_df['AvgPointsAgainst']

    # Reset the index
    summary_df.reset_index(inplace=True)

    # Add a new column that indicates the subcategory
    summary_df['GameSubcategory'] = df_name

    # Add the summary DataFrame to the list
    summary_dfs.append(summary_df)

# Concatenate all the summary DataFrames together
final_summary_df = pd.concat(summary_dfs)

In [13]:
final_summary_df.sample(10)

,SeasonYear,SportCode,TeamName,TotalGames,ResultTotal,Wins,Losses,Ties,AvgPointsFor,TotalPointsFor,AvgPointsAgainst,TotalPointsAgainst,WinPct,AvgMargin,GameSubcategory
43943,2018,BBA,Blanchard Montabella,10,10,0,10,0,3.800000,38,10.800000,108,0.000000,-7.000000,AllGames
45155,2018,GBB,Grand Rapids Homeschoolers,2,2,0,2,0,16.000000,32,37.000000,74,0.000000,-21.000000,RegularSeason
7512,2012,GBB,Rochester,1,1,0,1,0,48.000000,48,55.000000,55,0.000000,-7.000000,DistrictGames
20081,2015,BSO,Niles Brandywine,1,1,0,1,0,1.000000,1,4.000000,4,0.000000,-3.000000,Postseason
31656,2015,GSO,Traverse City St Francis (3 of 3),2,2,0,0,2,0.000000,0,0.000000,0,0.000000,0.000000,RegularSeason
12426,2014,BBA,Flint Beecher,1,1,0,1,0,0.000000,0,1.000000,1,0.000000,-1.000000,DistrictGames
19785,2013,BFB,Walled Lake Western,9,9,8,1,0,39.111111,352,22.111111,199,0.888889,17.000000,RegularSeason
50450,2020,GSB,Vermontville Maple Valley,8,8,0,8,0,1.125000,9,18.125000,145,0.000000,-17.000000,BlowoutGames
50985,2020,BBB,Lincoln Park,14,14,3,11,0,43.214286,605,49.571429,694,0.214286,-6.357143,RegularSeason
46325,2018,BFB,Swartz Creek,10,10,6,4,0,26.400000,264,26.100000,261,0.600000,0.300000,AllGames


In [14]:
### Output final summary to csv file

final_summary_df.to_csv('../data/all_sports_year_by_year_summary.csv', index=False)

In [12]:
ewqfqwefqw

NameError: name 'ewqfqwefqw' is not defined

In [ ]:
grouped = df.groupby(['SeasonYear', 'SportCode', 'SeasonType', 'GameType', 'PostSeasonLevel', 'PostSeasonSubLevel', 'TeamName'])
# Now you can group by several columns and calculate the aggregates
summary_df = grouped.agg({
    'HostSchoolId': 'count',  # Total games played
    'Result': ['count', lambda x: (x == 'W').sum(), lambda x: (x == 'L').sum(), lambda x: (x == 'T').sum()],  # Total wins, losses, and ties
    'TeamScore': ['mean', 'sum'],  # Average and total points for
    'OpponentScore': ['mean', 'sum'],  # Average and total points against
})


# Rename the columns to your liking
summary_df.columns = ['TotalGames', 'ResultTotal', 'Wins', 'Losses', 'Ties', 'AvgPointsFor', 'TotalPointsFor', 'AvgPointsAgainst', 'TotalPointsAgainst']



# Add other statistics like win percentage, average margin of victory/defeat, etc.
summary_df['WinPct'] = summary_df['Wins'] / summary_df['TotalGames']
summary_df['AvgMargin'] = summary_df['AvgPointsFor'] - summary_df['AvgPointsAgainst']

# # Reset the index to make 'SeasonYear', 'SportCode', 'SeasonType', and 'GameType' back to columns
summary_df.reset_index(inplace=True)



In [ ]:
summary_df.sample(20)

## SeasonType value counts
summary_df['SeasonType'].value_counts()

In [ ]:
df.columns

In [ ]:
### Save raw data to csv]
# master_df.to_csv(f'../data/TUES_Night_raw.csv', index=False)


In [ ]:
master_df.sample(10)

In [ ]:
## Define the slices of the data I want to aggrigate for calculations
## Slices will be denoted by creating a new column called SeasonType That will hold the value of the slice (Overall, Regular Season, Postseason, etc.)

### Slices - For Each Sport and Each year
## all games,
## Home and away, 
## regular season and postseason, 
## overtime_games, 
## level of postseason - District / Regional / State
## Close games and blowout games - (have a dictionary that defines the threasholds for each sport) 
score_threasolds = {
    'BIH': {"close": 1, "blowout": 5},
    'BFB': {"close": 7, "blowout": 21},
    'BBA': {"close": 3, "blowout": 8},
    'GSB': {"close": 3, "blowout": 8},
    'BBB': {"close": 3, "blowout": 15},
    'GBB': {"close": 3, "blowout": 15},
    'BBL': {"close": 2, "blowout": 8},
    'GGL': {"close": 2, "blowout": 8},
    'BSO': {"close": 1, "blowout": 5},
    'GSO': {"close": 1, "blowout": 5},
}



## want to calculate the following states for each situation for every year and sport.
# I'd also like to create a row that represents the overall stats for each team for each sport
# Total games played
# Total wins loses and ties
# win percentage
# average points for
# average points against
# average margin of victory/defeat



In [ ]:
wergwerg

In [ ]:
######## Display master dataframe ########

master_df.head()

master_df.tail()

# master_df.info()

# master_df.describe()

master_df.columns

master_df.shape

# master_df.to_csv('master_df_v3.csv')


In [ ]:


#######  DICTIONARY OF WHAT IS A CLOSE AND BLOWOUT GAME FOR EACH SPORT ############################

score_threasolds = {
    'BIH': {"close": 1, "blowout": 5},
    'BFB': {"close": 7, "blowout": 21},
    'BB': {"close": 3, "blowout": 8},
    'GSB': {"close": 3, "blowout": 8},
    'BBB': {"close": 3, "blowout": 15},
    'GBB': {"close": 3, "blowout": 15},
    'BBL': {"close": 2, "blowout": 8},
    'GGL': {"close": 2, "blowout": 8},
    'BSO': {"close": 1, "blowout": 5},
    'GSO': {"close": 1, "blowout": 5},
}




## Create a new dataframe column for game situation
df['GameSituation'] = df.apply(get_game_situation, axis=1)

# Seperate close and blowout games
close_games = df[df['GameSituation'] == 'Close']
blowout_games = df[df['GameSituation'] == 'Blowout']


## Define the Slices for Game Situation Catagories

# Overall (all regular and post season games)
overall = df

# Regular Season
regular_season = df[df['IsPostSeason'] == False]

# Post Season
post_season = df[df['IsPostSeason'] == True]
# If the scores are 0 drop the row
post_season = post_season.drop(post_season[(post_season['HostScore'] == 0) & (post_season['AwayScore'] == 0)].index)

# Overtime Games
overtime_games = df[df['OT'].notnull()]

# Assuming 'PostSeasonLevel' is a column with values like 'District', 'Regional', etc.
district_games = df[df['PostSeasonLevel'] == 'District']
regional_games = df[df['PostSeasonLevel'] == 'Regional']
quarterfinal_games = df[df['PostSeasonLevel'] == 'Quarterfinal']
semifinal_games = df[df['PostSeasonLevel'] == 'Semifinal']
final_games = df[df['PostSeasonLevel'] == 'Final']

### Create new dataframes organized by team of all home games and all away games
df_home = regular_season[['SeasonYear', 'SportName', 'StartDate', 'IsPostSeason', 'Notes',
        'OT', 'ScoreNotes', 'HostSchoolId', 'HostScore', 'HostTeamName',
            'ContestNotes', 'TournamentNumber', 'PostSeasonLevel', 'SportCode',
            'AwaySchoolId', 'AwayScore', 'AwayTeamName', 'HostResult']]
## Assign 'SeasonType' as 'Home'
df_home['SeasonType'] = 'Home'

df_away = regular_season[['SeasonYear', 'SportName', 'StartDate', 'IsPostSeason', 'Notes',
            'OT', 'ScoreNotes', 'HostSchoolId', 'HostScore', 'HostTeamName',
            'ContestNotes', 'TournamentNumber', 'PostSeasonLevel', 'SportCode',
            'AwaySchoolId', 'AwayScore', 'AwayTeamName', 'AwayResult']]
## Assign 'SeasonType' as 'Away'
df_away['SeasonType'] = 'Away'


In [ ]:
asdfgasdf

In [ ]:
df.info()

In [ ]:
######### see if anything came out of it

print(df_all.shape)

print(df_all.info())
# df_all.sample(10)
df_all.head()


In [ ]:

# Copied to above



# def yearly_stats(df):
#     # drop any forfieted games (score = 0)
#     df = df.loc[(df['HostScore'] != 0) & (df['AwayScore'] != 0)]

#     #### Drop null scores
#     df = df.dropna(subset=['HostScore', 'AwayScore'])
#     ## scores to int
#     df['HostScore'] = df['HostScore'].astype(int)
#     df['AwayScore'] = df['AwayScore'].astype(int)


#     # Create new columns holding the result for the home and away teams
#     df['HostResult'] = np.where(df['HostScore'] > df['AwayScore'], 'W', 
#                                 np.where(df['HostScore'] < df['AwayScore'], 'L', 
#                                         np.where(df['HostScore'] == df['AwayScore'], 'T', None)))

#     df['AwayResult'] = np.where(df['HostScore'] > df['AwayScore'], 'L', 
#                                 np.where(df['HostScore'] < df['AwayScore'], 'W', 
#                                         np.where(df['HostScore'] == df['AwayScore'], 'T', None)))



#     #######  DICTIONARY OF WHAT IS A CLOSE AND BLOWOUT GAME FOR EACH SPORT ############################

#     score_threasolds = {
#         'BIH': {"close": 1, "blowout": 5},
#         'BFB': {"close": 7, "blowout": 21},
#         'BSB': {"close": 3, "blowout": 8},
#         'GSB': {"close": 3, "blowout": 8},
#         'BBB': {"close": 3, "blowout": 15},
#         'GBB': {"close": 3, "blowout": 15},
#         'BBL': {"close": 2, "blowout": 8},
#         'GGL': {"close": 2, "blowout": 8},
#         'BSO': {"close": 1, "blowout": 5},
#         'GSO': {"close": 1, "blowout": 5},
#     }




#     ## Create a new dataframe column for game situation
#     df['GameSituation'] = df.apply(get_game_situation, axis=1)

#     # Seperate close and blowout games
#     close_games = df[df['GameSituation'] == 'Close']
#     blowout_games = df[df['GameSituation'] == 'Blowout']


#     ## Define the Slices for Game Situation Catagories

#     # Overall (all regular and post season games)
#     overall = df

#     # Regular Season
#     regular_season = df[df['IsPostSeason'] == False]

#     # Post Season
#     post_season = df[df['IsPostSeason'] == True]
#     # If the scores are 0 drop the row
#     post_season = post_season.drop(post_season[(post_season['HostScore'] == 0) & (post_season['AwayScore'] == 0)].index)

#     # Overtime Games
#     overtime_games = df[df['OT'].notnull()]

#     # Assuming 'PostSeasonLevel' is a column with values like 'District', 'Regional', etc.
#     district_games = df[df['PostSeasonLevel'] == 'District']
#     regional_games = df[df['PostSeasonLevel'] == 'Regional']
#     quarterfinal_games = df[df['PostSeasonLevel'] == 'Quarterfinal']
#     semifinal_games = df[df['PostSeasonLevel'] == 'Semifinal']
#     final_games = df[df['PostSeasonLevel'] == 'Final']

#     ### Create new dataframes organized by team of all home games and all away games
#     df_home = regular_season[['SeasonYear', 'SportName', 'StartDate', 'IsPostSeason', 'Notes',
#             'OT', 'ScoreNotes', 'HostSchoolId', 'HostScore', 'HostTeamName',
#                 'ContestNotes', 'TournamentNumber', 'PostSeasonLevel', 'SportCode',
#                 'AwaySchoolId', 'AwayScore', 'AwayTeamName', 'HostResult']]
#     ## Assign 'SeasonType' as 'Home'
#     df_home['SeasonType'] = 'Home'

#     df_away = regular_season[['SeasonYear', 'SportName', 'StartDate', 'IsPostSeason', 'Notes',
#                 'OT', 'ScoreNotes', 'HostSchoolId', 'HostScore', 'HostTeamName',
#                 'ContestNotes', 'TournamentNumber', 'PostSeasonLevel', 'SportCode',
#                 'AwaySchoolId', 'AwayScore', 'AwayTeamName', 'AwayResult']]
#     ## Assign 'SeasonType' as 'Away'
#     df_away['SeasonType'] = 'Away'



#     ### Append all the dataframes together and save to a new csv
#     df_all = pd.DataFrame()
#     for df, name in dataframes:
#         df_all = df_all.append(df)


In [ ]:
##### Show stats for df_all
    print('Overall Stats')
    print('Total Games: ', len(df_all))
    print('Total Home Wins: ', len(df_all[df_all['HostResult'] == 'W']))
    print('Total Home Losses: ', len(df_all[df_all['HostResult'] == 'L']))
